In [1]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu   # jeśli nie masz GPU, to wersja CPU jest lżejsza i szybsza w instalacji
#!pip install pytorch-lightning pandas py3Dmol mols2grid selfies joblib scikit-learn rdkit-pypi --quiet

In [ ]:
import rdkit, torch, selfies, mols2grid, pytorch_lightning as pl
import random, warnings, os
import pandas as pd
import numpy as np
import selfies as sf
from rdkit import Chem
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
from rdkit.Chem import QED
print("RDKit:", rdkit.__version__)
print("Torch:", torch.__version__)
print("Selfies check:", selfies.encoder("CCO")) #because of errors in previous attempt

In [ ]:
import random, warnings, os
import pandas as pd
import numpy as np
import selfies as sf
from rdkit import Chem
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from sklearn.ensemble import RandomForestRegressor
from joblib import dump
from rdkit.Chem import QED
import mols2grid
from joblib import load

# downloading files
url = "https://raw.githubusercontent.com/asapdiscovery/COVID_moonshot_data_clean/main/cdd_noncovalent_dates_2023_10_18_filt.csv"
df = pd.read_csv(url)

def clean_ic50(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    if s.startswith('>'): return float(s[1:])
    if s.startswith('<'): return float(s[1:]) / 10
    try: return float(s)
    except: return np.nan

df['IC50_clean'] = df['IC50_(µM)'].apply(clean_ic50)
df = df.dropna(subset=['suspected_SMILES', 'IC50_clean'])
df = df[df['IC50_clean'] <= 1000] # rozsądny zakres
df['pIC50'] = -np.log10(df['IC50_clean'] * 1e-6)
print(f"After data processing num of molecules {len(df)} (pIC50 the lowest {df['pIC50'].min():.2f} to highest {df['pIC50'].max():.2f})")

# Fingerprint – z lepszym fallbackiem, bo niektóre SMILESy się wywalają
def get_fp(smiles):
    mol = Chem.MolFromSmiles(smiles, sanitize=True)
    if mol:
        return GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    # cleaning, because SMILES returns errors
    clean = ''.join(c for c in smiles if c.isalpha() or c in '()=#1234567890')
    mol = Chem.MolFromSmiles(clean, sanitize=False)
    if mol:
        try:
            Chem.SanitizeMol(mol)
            return GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        except:
            pass
    return np.zeros(2048, dtype=np.float32)

print("Generating fingerprints")
X = [get_fp(sm) for sm in df['suspected_SMILES']]
y = df['pIC50'].values

print("Training random forest")
model = RandomForestRegressor(n_estimators=600, n_jobs=-1, random_state=42)
model.fit(X, y)

# Save model for the next steps
dump(model, 'my_mpro_model.pkl')
print("Model saved → my_mpro_model.pkl")

# Test to check on Nirmatrelvir- Paxlovid
nirmatrelvir_smiles = "CC(C)(C)[C@@H]1C(=O)N[C@H](C(=O)N2C[C@H]3[C@@H](C2)C(=O)N3)CC4=CC=CC=C4"
pred = model.predict([get_fp(nirmatrelvir_smiles)])[0]
print(f"Nirmatrelvir (Paxlovid) – prediction pIC50: {pred:.3f} (real ~9.0–9.3)")

In [ ]:
warnings.filterwarnings("ignore")

# load model
model = load("my_mpro_model.pkl")

# Bezpieczny fingerprint – ten sam co wyżej
def get_fp_safe(smiles):
    mol = Chem.MolFromSmiles(smiles, sanitize=True)
    if mol:
        return GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    clean = ''.join(c for c in smiles if c.isalpha() or c in '()=#1234567890')
    mol = Chem.MolFromSmiles(clean, sanitize=False)
    if mol:
        try:
            Chem.SanitizeMol(mol)
            return GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        except:
            pass
    return np.zeros(2048, dtype=np.float32)

def score_smiles(smiles):
    fp = get_fp_safe(smiles)
    return float(model.predict([fp])[0])

# SELFIES mutation generate – poprawione importy
from selfies.utils import mutator, crossover

def mutate(smiles):
    try:
        selfie = sf.encoder(smiles)
        mutated = mutator(selfie)
        return sf.decoder(mutated)
    except:
        return smiles

def crossover(a, b):
    try:
        return sf.decoder(crossover(sf.encoder(a), sf.encoder(b)))
    except:
        return random.choice([a, b])

# molecule population in the beginning from CSV
if not os.path.exists("zinc.csv"):
    os.system('wget -q -O zinc.csv "https://raw.githubusercontent.com/aspuru-guzik-group/chemical_vae/master/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv"')

df_zinc = pd.read_csv("zinc.csv")
start_smiles = df_zinc["smiles"].dropna().sample(100000, random_state=42).tolist()

POP = 1200
KIDS = 2400
GENS = 1200

print("Building starting population")
population = []
for sm in random.sample(start_smiles, 12000):
    sc = score_smiles(sm)
    if sc > 5.0:
        population.append((sm, sc))

population = sorted(population, key=lambda x: x[1], reverse=True)[:POP]
print(f"Start – the best pIC50: {population[0][1]:.3f}")

best_ever = population[0][1]

for gen in range(GENS):
    scores = np.array([s for _,s in population])
    probs = np.exp((scores - scores.max()) / 0.18)
    probs /= probs.sum()
    parents = random.choices(population, weights=probs, k=KIDS)
    
    children = []
    for i in range(0, KIDS-1, 2):
        p1, p2 = parents[i][0], parents[i+1][0]
       
        child = mutate(p1) if random.random() < 0.5 else crossover(p1, p2)
        if random.random() < 0.4:
            child = mutate(child)
            
        sc = score_smiles(child)
        mol = Chem.MolFromSmiles(child)
        if not mol: continue
        qed = QED.qed(mol)
        final = sc * 1.9 + qed * 1.4
        
        if final > best_ever + 0.08:
            best_ever = final
            print(f"\nRECORD! gen {gen} | pIC50 ≈ {sc:.3f} | final {final:.3f}")
            print(child)
            
        children.append((child, final))
        
    population += children
    population = sorted(population, key=lambda x: x[1], reverse=True)[:POP]
    
    # Raport every 50 generation because it sometimes take too much time so good to know about progress
    if gen % 50 == 0:
        best = population[0]
        print(f"Gen {gen:4d} | pIC50 ≈ {score_smiles(best[0]):.3f} | atoms {Chem.MolFromSmiles(best[0]).GetNumAtoms()}")
        
    # For safety saving every 200 generation
    if gen % 200 == 0 and gen > 0:
        top20 = population[:20]
        pd.DataFrame(top20, columns=['SMILES', 'final_score']).to_csv(f"best_gen_{gen}.csv", index=False)
        print(f" TOP 20 save to file best_gen_{gen}.csv")

print("\n End the best TOP20:")
top_mols = [Chem.MolFromSmiles(s) for s, _ in population[:20] if Chem.MolFromSmiles(s)]
mols2grid.display(top_mols, n_cols=5, size=(260,200))

# Save everything at the end
pd.DataFrame(population[:100], columns=['SMILES', 'final_score']).to_csv("FINAL_TOP100.csv", index=False)
print("FINAL_TOP100.csv is saving!")